## Backtesting Sesion 1
### Evaluación de Señales en Ventanas Deslizantes

Este cuaderno muestra un ejemplo de como realizar una evaluación fuera de los datos de optimización, utilizando ventanas deslizantes.  Las ideas generales las podemos resumir en:
- Escoger un conjunto de parámetros a partir de la exploración sobre un periodo de tiempo puede llevar a un sobre-ajuste de la estrategia
- Una evaluación más realista consiste calcular el rendimiento en un periodo diferente al de optimización
- Podemos simular un periodo continuo de evaluación concatenando ventanas deslizantes de entrenamiento/prueba 


In [ ]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import itertools

____
### Datos 
Snapshot de acciones del IBEX35

In [ ]:
import pickle
with open('../data/stock_data.pkl', 'rb') as handle:
    stock_data = pickle.load(handle)

Generamos un único dataframe con los precios de cierre

In [ ]:
close_dict = {tk: df.close for tk,df in stock_data.items()}
stock_close = pd.DataFrame(close_dict)
stock_close.head()

Trabajaremos primero con un solo valor, que podemos ir variando

In [ ]:
ticker = 'TEF'
stock_series = stock_close[ticker].dropna()

In [ ]:
stock_series.plot()

___

In [ ]:
from stoosc import Sto # clase con las funciones del oscilador estocastico

In [ ]:
Sto.backtest_so_returns(stock_series, win=25, obought=0.8, osold=0.2)

Conjunto de parámetros para la exploración. (Igual que en cuaderno 1.1)

In [ ]:
obought_params = [0.70, 0.80, 0.90]
osold_params = [0.10, 0.20, 0.30]
win_params = [20, 30, 50]
combined_params = list(itertools.product(win_params, obought_params, osold_params))
combined_params[:10]

____
### Función de Exploración 
Reutilizamos código de exploración para tener una función que dado un espacio de parámetros y una serie, nos devuelva
la mejor rentabilidad obtenida, y la combinación que lo produce

In [ ]:
?Sto.backtest_so_returns

In [ ]:
def explore_sto_params(params_product, vseries):
    result = {}
    for iparams in params_product:
        (w, b, s) = iparams
        result[iparams] = Sto.backtest_so_returns(vseries, win=w, obought=b, osold=s)
    rseries = pd.Series(result)
    return rseries.idxmax(), rseries.max()

In [ ]:
subserie = stock_series.loc[:'2005']
subserie

In [ ]:
explore_sto_params(combined_params, subserie)

### Ventanas

In [ ]:
years = list(np.arange(2003,2021))
years

___
### Diseño Experimental
- Utilizar 2 años para determinar la mejor combinación de parámetros
- Utilizamos la mejor combinación en los 3 años posteriores para evaluar el rendimiento.

Este enfoque más realista porque si tuvieramos que decidir **hoy** que parámetros elegir, 
podemos elegir la combinación del pasado, pero nuestro resultado real es el que conseguiríamos
a partir de aquí en adelante

In [ ]:
# Separacion en años
for i in range(len(years) - 4):
    print(years[i:i+2], years[i+2:i+5])

___
Agregamos el paso de exploración.  En cada iteración determinamos el mejor conjunto de parámetros

In [ ]:
for i in range(len(years) - 4):
    fityears = years[i:i+2]
    fitseries = stock_series.loc[f'{fityears[0]}':f'{fityears[-1]}']
    best_params, best_ret = explore_sto_params(combined_params, fitseries)
    print(fityears, ':', best_params, np.round(best_ret,3))

____
incluimos la evaluación fuera de los años de ajuste

In [ ]:
fit_rets = []
test_rets = []
for i in range(len(years) - 4):
    fityears = years[i:i+2]
    fitseries = stock_series.loc[f'{fityears[0]}':f'{fityears[-1]}']
   
    best_params, best_ret = explore_sto_params(combined_params, fitseries)
    print(fityears, ':', best_params,np.round(best_ret,3))
    fit_rets.append(best_ret)

    # Ejecutamos la funcion del backtesting para calcular resultado con los parametros seleccionados
    testyears = years[i+2:i+5]
    testseries = stock_series.loc[f'{testyears[0]}':f'{testyears[-1]}']
    
    w, b, s = best_params
    test_ret = Sto.backtest_so_returns(testseries, win=w, obought=b, osold=s)
    print("--> Test:", testyears, best_params, np.round(test_ret,3))
    test_rets.append(test_ret)

____

juntamos rendimientos para visualización

In [ ]:
df_rets = pd.DataFrame({
    'fit': fit_rets,
    'test': test_rets,
})
df_rets.plot()

In [ ]:
df_rets.describe()

### Observaciones 
- El resultado muestra que la selección del mejor conjunto de parámetros produce sobre-ajuste
- El rendimiento esperado es el que corresponde al período de prueba

___
### Ejercicios Propuestos
- Determinar si el tamaño de la ventana de entrenamiento influye en la diferencia de rendimiento entre fit y test. Sugerencia:
  - Extender diseño experimental para incrementar años de ajuste de 1 a 4 (ver código más abajo)
  - Guardar resultados por grupos de tamaño de ajuste

- Confirmar conclusiones utilizando otra serie de precios

In [ ]:
for yd in range(4):
    for i in range(len(years) - 3 - yd):
        print(years[i:i+1+yd], years[i+1+yd:i+4+yd])